In [1]:
import os
import pandas as pd
import librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

## Data

In [8]:
# sample_rate_dataframe = pd.DataFrame(columns=["File_Name", "Sample_Rate", "Duration"])
# for item in data_list:
#     sample_rate = librosa.get_samplerate(item)
#     duration = librosa.get_duration(y=item, sr=sample_rate)
#     new_row = {"File_Name": item,"Sample_Rate": sample_rate, "Duration": duration}
#     sample_rate_dataframe = pd.concat(
#             [sample_rate_dataframe, pd.DataFrame(new_row, index=[0])])
# sample_rate_dataframe = sample_rate_dataframe.reset_index(drop=True)

# print(sample_rate_dataframe.Duration.min())
# print(sample_rate_dataframe.Duration.max())
# print(sample_rate_dataframe.Duration.mean())
# print(sample_rate_dataframe.Sample_Rate.value_counts())


AttributeError: 'str' object has no attribute 'shape'

In [2]:
data_list = []
folder_file = os.listdir("ravdess_dataset/audio_speech_actors_01-24")
for x in folder_file:
    item_file = os.listdir("ravdess_dataset/audio_speech_actors_01-24/{}".format(x))
    for y in item_file:
        data_list.append("ravdess_dataset/audio_speech_actors_01-24/{}/{}".format(x,y))

labels = {'disgust': 0, 'happy': 1, 'sad': 2,
            'neutral': 3, 'fear': 4, 'angry': 5}
data_sentiment_path = []
data_sentiment_value = []
data_sentiment_encoded_value = []
for file in data_list:
    sentiment_code = file[-18:-16]
    if sentiment_code == '05':
        data_sentiment_path.append(file)
        data_sentiment_value.append('angry')  
        data_sentiment_encoded_value.append(5)        
    elif sentiment_code == '07':
        data_sentiment_path.append(file)
        data_sentiment_value.append('disgust')  
        data_sentiment_encoded_value.append(0)
    elif sentiment_code == '06':
        data_sentiment_path.append(file)
        data_sentiment_value.append('fear')  
        data_sentiment_encoded_value.append(4)
    elif sentiment_code == '03':
        data_sentiment_path.append(file)
        data_sentiment_value.append('happy')  
        data_sentiment_encoded_value.append(1)
    elif sentiment_code == '01':
        data_sentiment_path.append(file)
        data_sentiment_value.append('neutral')  
        data_sentiment_encoded_value.append(3)
    elif sentiment_code == '04':
        data_sentiment_path.append(file)
        data_sentiment_value.append('sad')  
        data_sentiment_encoded_value.append(2)
    else:
        pass

## Original Dataset

In [3]:
data_list = []
train_files = []
test_files = []

folder_file = os.listdir("ravdess_dataset/audio_speech_actors_01-24")
for x in folder_file:
    item_file = os.listdir("ravdess_dataset/audio_speech_actors_01-24/{}".format(x))
    for y in item_file:
        data_list.append("ravdess_dataset/audio_speech_actors_01-24/{}/{}".format(x,y))
        
train_data_sentiment_path = []
train_data_sentiment_value = []
train_data_sentiment_encoded_value = []

test_data_sentiment_path = []
test_data_sentiment_value = []
test_data_sentiment_encoded_value = []

import random
random.seed(1234)
for file_path in data_list:
    if random.random() < 0.8:
        train_files.append(file_path)
    else:
        test_files.append(file_path)

for file in train_files:
    sentiment_code = file[-18:-16]
    if sentiment_code == '05':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('angry')  
        train_data_sentiment_encoded_value.append(5)        
    elif sentiment_code == '07':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('disgust')  
        train_data_sentiment_encoded_value.append(0)
    elif sentiment_code == '06':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('fear')  
        train_data_sentiment_encoded_value.append(4)
    elif sentiment_code == '03':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('happy')  
        train_data_sentiment_encoded_value.append(1)
    elif sentiment_code == '01':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('neutral')  
        train_data_sentiment_encoded_value.append(3)
    elif sentiment_code == '04':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('sad')  
        train_data_sentiment_encoded_value.append(2)
    else:
        pass

for file in test_files:
    sentiment_code = file[-18:-16]
    if sentiment_code == '05':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('angry')  
        test_data_sentiment_encoded_value.append(5)        
    elif sentiment_code == '07':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('disgust')  
        test_data_sentiment_encoded_value.append(0)
    elif sentiment_code == '06':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('fear')  
        test_data_sentiment_encoded_value.append(4)
    elif sentiment_code == '03':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('happy')  
        test_data_sentiment_encoded_value.append(1)
    elif sentiment_code == '01':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('neutral')  
        test_data_sentiment_encoded_value.append(3)
    elif sentiment_code == '04':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('sad')  
        test_data_sentiment_encoded_value.append(2)
    else:
        pass

num_mfcc = 40
SAMPLE_RATE = 48000
n_fft = 2048
hop_length = 512

train_data = {
    "labels": [],
    "mfcc": []
}

test_data = {
    "labels": [],
    "mfcc": []
}

for path, value in zip(train_data_sentiment_path, train_data_sentiment_encoded_value):
    signal, sample_rate = librosa.load(path)
    mfcc = librosa.feature.mfcc(
        y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    mfcc = mfcc.T
    
    train_data['labels'].append(value)
    train_data["mfcc"].append(np.asarray(mfcc))

    # AugmentType = "PitchShift"
    # signal, sample_rate = librosa.load(path.replace("audio_speech_actors_01-24", "Augment/{}".format(AugmentType)))
    # mfcc = librosa.feature.mfcc(
    #     y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    # mfcc = mfcc.T
    # train_data['labels'].append(value)
    # train_data["mfcc"].append(np.asarray(mfcc))


for path, value in zip(test_data_sentiment_path, test_data_sentiment_encoded_value):
    signal, sample_rate = librosa.load(path)
    mfcc = librosa.feature.mfcc(
        y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    mfcc = mfcc.T
    
    test_data['labels'].append(value)
    test_data["mfcc"].append(np.asarray(mfcc))



processed_data_value = np.asarray(train_data['mfcc'])
processed_data_target = np.asarray(train_data["labels"])
processed_test_value = np.asarray(test_data['mfcc'])
processed_test_target = np.asarray(test_data["labels"])

padded_data_value = tf.keras.preprocessing.sequence.pad_sequences(
    processed_data_value, maxlen=228 ,dtype="float32")
padded_test_value = tf.keras.preprocessing.sequence.pad_sequences(
    processed_test_value, maxlen=228 ,dtype="float32")

np.save('saved_dataset/ravdess_data.npy', padded_data_value)
np.save('saved_dataset/ravdess_data_target.npy', processed_data_target)
# np.save('saved_dataset/ravdess_test.npy', padded_test_value)
# np.save('saved_dataset/ravdess_test_target.npy', processed_test_target)

C:\Users\USER\AppData\Local\Temp\ipykernel_20956\3293100039.py:129: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  processed_data_value = np.asarray(train_data['mfcc'])
C:\Users\USER\AppData\Local\Temp\ipykernel_20956\3293100039.py:131: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  processed_test_value = np.asarray(test_data['mfcc'])


In [14]:
processed_data_target.shape

(847,)

## PitchShift Dataset

In [41]:
data_list = []
train_files = []
test_files = []

folder_file = os.listdir("ravdess_dataset/audio_speech_actors_01-24")
for x in folder_file:
    item_file = os.listdir("ravdess_dataset/audio_speech_actors_01-24/{}".format(x))
    for y in item_file:
        data_list.append("ravdess_dataset/audio_speech_actors_01-24/{}/{}".format(x,y))
        
train_data_sentiment_path = []
train_data_sentiment_value = []
train_data_sentiment_encoded_value = []

test_data_sentiment_path = []
test_data_sentiment_value = []
test_data_sentiment_encoded_value = []

import random
random.seed(1234)
for file_path in data_list:
    if random.random() < 0.8:
        train_files.append(file_path)
    else:
        test_files.append(file_path)

for file in train_files:
    sentiment_code = file[-18:-16]
    if sentiment_code == '05':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('angry')  
        train_data_sentiment_encoded_value.append(5)        
    elif sentiment_code == '07':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('disgust')  
        train_data_sentiment_encoded_value.append(0)
    elif sentiment_code == '06':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('fear')  
        train_data_sentiment_encoded_value.append(4)
    elif sentiment_code == '03':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('happy')  
        train_data_sentiment_encoded_value.append(1)
    elif sentiment_code == '01':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('neutral')  
        train_data_sentiment_encoded_value.append(3)
    elif sentiment_code == '04':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('sad')  
        train_data_sentiment_encoded_value.append(2)
    else:
        pass

for file in test_files:
    sentiment_code = file[-18:-16]
    if sentiment_code == '05':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('angry')  
        test_data_sentiment_encoded_value.append(5)        
    elif sentiment_code == '07':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('disgust')  
        test_data_sentiment_encoded_value.append(0)
    elif sentiment_code == '06':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('fear')  
        test_data_sentiment_encoded_value.append(4)
    elif sentiment_code == '03':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('happy')  
        test_data_sentiment_encoded_value.append(1)
    elif sentiment_code == '01':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('neutral')  
        test_data_sentiment_encoded_value.append(3)
    elif sentiment_code == '04':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('sad')  
        test_data_sentiment_encoded_value.append(2)
    else:
        pass

num_mfcc = 40
SAMPLE_RATE = 48000
n_fft = 2048
hop_length = 512

train_data = {
    "labels": [],
    "mfcc": []
}

test_data = {
    "labels": [],
    "mfcc": []
}

for path, value in zip(train_data_sentiment_path, train_data_sentiment_encoded_value):
    signal, sample_rate = librosa.load(path)
    mfcc = librosa.feature.mfcc(
        y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    mfcc = mfcc.T
    
    train_data['labels'].append(value)
    train_data["mfcc"].append(np.asarray(mfcc))

    AugmentType = "PitchShift"
    signal, sample_rate = librosa.load(path.replace("audio_speech_actors_01-24", "Augment/{}".format(AugmentType)))
    mfcc = librosa.feature.mfcc(
        y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    mfcc = mfcc.T
    train_data['labels'].append(value)
    train_data["mfcc"].append(np.asarray(mfcc))


for path, value in zip(test_data_sentiment_path, test_data_sentiment_encoded_value):
    signal, sample_rate = librosa.load(path)
    mfcc = librosa.feature.mfcc(
        y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    mfcc = mfcc.T
    
    test_data['labels'].append(value)
    test_data["mfcc"].append(np.asarray(mfcc))



processed_data_value = np.asarray(train_data['mfcc'])
processed_data_target = np.asarray(train_data["labels"])
processed_test_value = np.asarray(test_data['mfcc'])
processed_test_target = np.asarray(test_data["labels"])

padded_data_value = tf.keras.preprocessing.sequence.pad_sequences(
    processed_data_value, maxlen=228 ,dtype="float32")
padded_test_value = tf.keras.preprocessing.sequence.pad_sequences(
    processed_test_value, maxlen=228 ,dtype="float32")

np.save('saved_dataset/ravdess_dataWithPitchShift_train.npy', padded_data_value)
np.save('saved_dataset/ravdess_dataWithPitchShift_target.npy', processed_data_target)
np.save('saved_dataset/ravdess_test.npy', padded_test_value)
np.save('saved_dataset/ravdess_test_target.npy', processed_test_target)
    

C:\Users\USER\AppData\Local\Temp\ipykernel_13284\676355800.py:122: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  processed_data_value = np.asarray(train_data['mfcc'])
C:\Users\USER\AppData\Local\Temp\ipykernel_13284\676355800.py:124: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  processed_test_value = np.asarray(test_data['mfcc'])


## TimeStretch Dataset

In [4]:
data_list = []
train_files = []
test_files = []

folder_file = os.listdir("ravdess_dataset/audio_speech_actors_01-24")
for x in folder_file:
    item_file = os.listdir("ravdess_dataset/audio_speech_actors_01-24/{}".format(x))
    for y in item_file:
        data_list.append("ravdess_dataset/audio_speech_actors_01-24/{}/{}".format(x,y))

train_data_sentiment_path = []
train_data_sentiment_value = []
train_data_sentiment_encoded_value = []

test_data_sentiment_path = []
test_data_sentiment_value = []
test_data_sentiment_encoded_value = []

import random
random.seed(1234)
for file_path in data_list:
    if random.random() < 0.8:
        train_files.append(file_path)
    else:
        test_files.append(file_path)

for file in train_files:
    sentiment_code = file[-18:-16]
    if sentiment_code == '05':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('angry')  
        train_data_sentiment_encoded_value.append(5)        
    elif sentiment_code == '07':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('disgust')  
        train_data_sentiment_encoded_value.append(0)
    elif sentiment_code == '06':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('fear')  
        train_data_sentiment_encoded_value.append(4)
    elif sentiment_code == '03':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('happy')  
        train_data_sentiment_encoded_value.append(1)
    elif sentiment_code == '01':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('neutral')  
        train_data_sentiment_encoded_value.append(3)
    elif sentiment_code == '04':
        train_data_sentiment_path.append(file)
        train_data_sentiment_value.append('sad')  
        train_data_sentiment_encoded_value.append(2)
    else:
        pass

for file in test_files:
    sentiment_code = file[-18:-16]
    if sentiment_code == '05':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('angry')  
        test_data_sentiment_encoded_value.append(5)        
    elif sentiment_code == '07':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('disgust')  
        test_data_sentiment_encoded_value.append(0)
    elif sentiment_code == '06':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('fear')  
        test_data_sentiment_encoded_value.append(4)
    elif sentiment_code == '03':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('happy')  
        test_data_sentiment_encoded_value.append(1)
    elif sentiment_code == '01':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('neutral')  
        test_data_sentiment_encoded_value.append(3)
    elif sentiment_code == '04':
        test_data_sentiment_path.append(file)
        test_data_sentiment_value.append('sad')  
        test_data_sentiment_encoded_value.append(2)
    else:
        pass

num_mfcc = 40
SAMPLE_RATE = 48000
n_fft = 2048
hop_length = 512

train_data = {
    "labels": [],
    "mfcc": []
}

test_data = {
    "labels": [],
    "mfcc": []
}

for path, value in zip(train_data_sentiment_path, train_data_sentiment_encoded_value):
    signal, sample_rate = librosa.load(path)
    mfcc = librosa.feature.mfcc(
        y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    mfcc = mfcc.T
    
    train_data['labels'].append(value)
    train_data["mfcc"].append(np.asarray(mfcc))

    AugmentType = "TimeStretch"
    signal, sample_rate = librosa.load(path.replace("audio_speech_actors_01-24", "Augment/{}".format(AugmentType)))
    mfcc = librosa.feature.mfcc(
        y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    mfcc = mfcc.T
    train_data['labels'].append(value)
    train_data["mfcc"].append(np.asarray(mfcc))


# for path, value in zip(test_data_sentiment_path, test_data_sentiment_encoded_value):
#     signal, sample_rate = librosa.load(path)
#     mfcc = librosa.feature.mfcc(
#         y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
#     mfcc = mfcc.T
    
#     test_data['labels'].append(value)
#     test_data["mfcc"].append(np.asarray(mfcc))



processed_data_value = np.asarray(train_data['mfcc'])
processed_data_target = np.asarray(train_data["labels"])
# processed_test_value = np.asarray(test_data['mfcc'])
# processed_test_target = np.asarray(test_data["labels"])

padded_data_value = tf.keras.preprocessing.sequence.pad_sequences(
    processed_data_value, maxlen=228 ,dtype="float32")
# padded_test_value = tf.keras.preprocessing.sequence.pad_sequences(
#     processed_test_value, maxlen=228 ,dtype="float32")

np.save('saved_dataset/ravdess_dataWithTimeStretch2_train.npy', padded_data_value)
np.save('saved_dataset/ravdess_dataWithTimeStretch2_target.npy', processed_data_target)
# np.save('saved_dataset/ravdess_test.npy', padded_test_value)
# np.save('saved_dataset/ravdess_test_target.npy', processed_test_target)
    

C:\Users\USER\AppData\Local\Temp\ipykernel_10648\3576176263.py:129: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  processed_data_value = np.asarray(train_data['mfcc'])


In [15]:
test = np.load('saved_dataset/ravdess_dataWithTimeStretch2_train.npy')
test.shape
# processed_data_target.shape

(1694, 228, 40)

## Else

In [32]:
# from sklearn.preprocessing import StandardScaler
# flat_data = np.concatenate((processed_data_value, processed_test_value), axis=0)
# flat_data = np.concatenate(processed_test_value)
# scaler = StandardScaler()
# scaler.fit(flat_data)
# padded_data = []
# for sub_array in processed_data_value:
#     # Scale the sub-array using the fitted StandardScaler
#     scaled_sub_array = scaler.transform(sub_array.reshape(-1, sub_array.shape[-1])).reshape(sub_array.shape)

#     # Append the padded sub-array to the output list
#     padded_data.append(scaled_sub_array)
# padded_data_value = tf.keras.preprocessing.sequence.pad_sequences(
#     padded_data, dtype="float32")

# padded_test = []
# for sub_array in processed_test_value:
#     # Scale the sub-array using the fitted StandardScaler
#     scaled_sub_array = scaler.transform(sub_array.reshape(-1, sub_array.shape[-1])).reshape(sub_array.shape)

#     # Append the padded sub-array to the output list
#     padded_test.append(scaled_sub_array)
# padded_test_value = tf.keras.preprocessing.sequence.pad_sequences(
#     padded_data, dtype="float32")



In [38]:
test_data = np.load('saved_dataset/ravdess_test_target.npy')
test_data.shape

(199,)

In [30]:
padded_data_value.shape

(1714, 228, 40)

In [29]:
import math
def load_to_dataframe(train_folder_path, test_folder_path):
    """
    Loads train and test data from the specified file folders, and returns them as pandas DataFrame objects.
    :param train_folder_path: The path to the folder containing the train data files.
    :type train_folder_path: str
    :param test_folder_path: The path to the folder containing the test data files.
    :type test_folder_path: str
    :return: A two pandas DataFrame objects, containing the train and test data, respectively.
    :rtype: pandas.core.frame.DataFrame, pandas.core.frame.DataFrame
    """
    train_path = 'dataset/train'
    test_path = 'dataset/test'
    train_dir_list = os.listdir(train_path)
    test_dir_list = os.listdir(test_path)

    train_sentiment_value = []
    test_sentiment_value = []
    train_file_path = []
    test_file_path = []

    for file in train_dir_list:
        train_file_path.append(train_path + '/' + file)
        sentiment_code = file.split('_')
        if sentiment_code[2] == 'ANG':
            train_sentiment_value.append('angry')
        elif sentiment_code[2] == 'DIS':
            train_sentiment_value.append('disgust')
        elif sentiment_code[2] == 'FEA':
            train_sentiment_value.append('fear')
        elif sentiment_code[2] == 'HAP':
            train_sentiment_value.append('happy')
        elif sentiment_code[2] == 'NEU':
            train_sentiment_value.append('neutral')
        elif sentiment_code[2] == 'SAD':
            train_sentiment_value.append('sad')
        else:
            train_sentiment_value.append('unknown')

    for file in test_dir_list:
        test_file_path.append(test_path + '/' + file)
        sentiment_code = file.split('_')
        if sentiment_code[2] == 'ANG':
            test_sentiment_value.append('angry')
        elif sentiment_code[2] == 'DIS':
            test_sentiment_value.append('disgust')
        elif sentiment_code[2] == 'FEA':
            test_sentiment_value.append('fear')
        elif sentiment_code[2] == 'HAP':
            test_sentiment_value.append('happy')
        elif sentiment_code[2] == 'NEU':
            test_sentiment_value.append('neutral')
        elif sentiment_code[2] == 'SAD':
            test_sentiment_value.append('sad')
        else:
            test_sentiment_value.append('unknown')

    train_sentiment_df = pd.DataFrame(
        {"File_Path": train_file_path, "Target": train_sentiment_value})

    test_sentiment_df = pd.DataFrame(
        {"File_Path": test_file_path, "Target": test_sentiment_value})

    return train_sentiment_df, test_sentiment_df

def turn_into_data_for_model(train_df, test_df, mfcc_number=40, fft=2048, hop=512):
    # Set Variable for MFCC
    num_mfcc = mfcc_number
    SAMPLE_RATE = 16000
    n_fft = fft
    hop_length = hop

    train_data = {
        "labels": [],
        "mfcc": []
    }

    test_data = {
        "labels": [],
        "mfcc": []
    }

    # Encode Categories
    labels = {'disgust': 0, 'happy': 1, 'sad': 2,
              'neutral': 3, 'fear': 4, 'angry': 5}
    train_df_encoded = train_df.replace({'Target': labels}, inplace=False)
    test_df_encoded = test_df.replace({'Target': labels}, inplace=False)

    for item, row in train_df.iterrows():
        train_data['labels'].append(train_df_encoded.iloc[item, 1])
        signal, sample_rate = librosa.load(
            train_df_encoded.iloc[item, 0], sr=SAMPLE_RATE)
        mfcc = librosa.feature.mfcc(
            y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
        mfcc = mfcc.T
        train_data["mfcc"].append(np.asarray(mfcc))
        if item % 300 == 0:
            print("Train Size:" + str(math.floor(item)))

    for item, row in test_df.iterrows():
        test_data['labels'].append(test_df_encoded.iloc[item, 1])
        signal, sample_rate = librosa.load(
            test_df_encoded.iloc[item, 0], sr=SAMPLE_RATE)
        mfcc = librosa.feature.mfcc(
            y=signal, sr=SAMPLE_RATE, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
        mfcc = mfcc.T
        test_data["mfcc"].append(np.asarray(mfcc))
        if item % 300 == 0:
            print("Test Size:" + str(math.floor(item)))

    train_data_value = np.asarray(train_data['mfcc'])
    train_data_target = np.asarray(train_data["labels"])
    # train_data_value = tf.keras.preprocessing.sequence.pad_sequences(
    #     train_data_value, maxlen=156)
    test_data_value = np.asarray(test_data['mfcc'])
    test_data_target = np.asarray(test_data["labels"])
    # test_data_value = tf.keras.preprocessing.sequence.pad_sequences(
    #     test_data_value, maxlen=156)
    print(train_data_value.shape)
    print(test_data_value.shape)

    return train_data_value, train_data_target, test_data_value, test_data_target

In [30]:
train_df, test_df = load_to_dataframe('dataset/train', 'dataset/test')
crema_train_data_value, crema_train_data_target, crema_test_data_value, crema_test_data_target = turn_into_data_for_model(
    train_df, test_df, 40, 2048, 512)

Train Size:0
Train Size:300
Train Size:600
Train Size:900
Train Size:1200
Train Size:1500
Train Size:1800
Train Size:2100
Train Size:2400
Train Size:2700
Train Size:3000
Train Size:3300
Train Size:3600
Train Size:3900
Train Size:4200
Train Size:4500
Train Size:4800
Train Size:5100
Train Size:5400
Train Size:5700
Test Size:0
Test Size:300
Test Size:600
Test Size:900
Test Size:1200
(5953,)
(1489,)


C:\Users\USER\AppData\Local\Temp\ipykernel_3432\1236230660.py:111: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_data_value = np.asarray(train_data['mfcc'])
C:\Users\USER\AppData\Local\Temp\ipykernel_3432\1236230660.py:115: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_data_value = np.asarray(test_data['mfcc'])


In [31]:
combined_data = np.concatenate((crema_train_data_value, crema_test_data_value), axis=0)
crema_flat_data = np.concatenate(combined_data)
crema_scaler = StandardScaler()
crema_scaler.fit(crema_flat_data)

StandardScaler()

In [43]:
crema_padded_data = []
crema_padded_test_data = []
for sub_array in crema_train_data_value:
    # Scale the sub-array using the fitted StandardScaler
    scaled_sub_array = scaler.transform(sub_array.reshape(-1, sub_array.shape[-1])).reshape(sub_array.shape)

    # Append the padded sub-array to the output list
    crema_padded_data.append(scaled_sub_array)
crema_padded_data_value = tf.keras.preprocessing.sequence.pad_sequences( 
    crema_padded_data, dtype="float32", maxlen=156)

for sub_array in crema_test_data_value:
    # Scale the sub-array using the fitted StandardScaler
    scaled_sub_array = scaler.transform(sub_array.reshape(-1, sub_array.shape[-1])).reshape(sub_array.shape)

    # Append the padded sub-array to the output list
    crema_padded_test_data.append(scaled_sub_array)
crema_padded_data_test_value = tf.keras.preprocessing.sequence.pad_sequences(
    crema_padded_test_data, dtype="float32", maxlen=156)

np.save('saved_dataset/crema_data_train.npy', crema_padded_data_value)
np.save('saved_dataset/crema_data_target.npy', crema_train_data_target)
np.save('saved_dataset/crema_data_test.npy', crema_padded_data_test_value)
np.save('saved_dataset/crema_data_test_target.npy', crema_test_data_target)


In [44]:
crema_padded_data_test_value.shape

(1489, 156, 40)

In [64]:
def vectorToMFCC(vector):
  vector.T
  librosa.display.specshow(vector, hop_length=512)
  plt.xlabel("Time")
  plt.ylabel("MFCC")
  plt.colorbar()
  plt.show()

## Gajelas

In [110]:
shape_processed = []
for x in processed_data_value:
    shape_processed.append(x.shape[0])

In [115]:
from statistics import mean
mean(shape_processed)

160.82291666666666

In [51]:
padded_data_value = tf.keras.preprocessing.sequence.pad_sequences(
    processed_data_value, dtype="float32")

In [36]:
processed_data_value[0]

array([[-1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00

In [28]:
padded_data_value.shape

(1056, 228, 40)

In [52]:
padded_data_value[0]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00

In [38]:
np.save("ravdess_data-value-normalized2", padded_data_value)
np.save("ravdess_data_target-normalized2", processed_data_target)

In [116]:
np.save("ravdess_data-value", padded_data_value)
np.save("ravdess_data_target", processed_data_target)

In [42]:
padded_data_value = np.load("ravdess_data-value.npy")

In [53]:
from sklearn.model_selection import train_test_split

train_data_value, test_data_value, train_data_target, test_data_target = train_test_split(padded_data_value, processed_data_target, test_size=0.2, random_state=42)


In [31]:
train_data_value[4]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   

In [30]:
test_data_value[4]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   

In [54]:
def cnn_lstm(optimizer='adam', learning_rate=0.001):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(
        padded_data_value.shape[1], padded_data_value.shape[2], 1)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D(
        (2, 2), strides=(2, 2), padding='same'))

    model.add(tf.keras.layers.Conv2D(64, (3, 3)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D(
        (2, 2), strides=(2, 2), padding='same'))

    model.add(tf.keras.layers.Conv2D(128, (2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))
    model.add(tf.keras.layers.MaxPooling2D(
        (2, 2), strides=(2, 2), padding='same'))

    model.add(tf.keras.layers.Conv2D(128, (2, 2)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation('elu'))

    model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)))
    model.add(tf.keras.layers.Dense(6, activation="softmax"))
    optimiser = tf.keras.optimizers.get(optimizer)
    optimiser.learning_rate.assign(learning_rate)
    model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    # model.summary()
    str_learning_rate = str(learning_rate).replace('.', '')
    # csv_logger = tf.keras.callbacks.CSVLogger(
    #     'robust_cnn_lstm{0}_{1}.csv'.format(optimizer, str_learning_rate))
    history = model.fit(train_data_value, train_data_target, validation_data=(
        test_data_value, test_data_target), batch_size=32, epochs=60)
    return model

In [41]:
cnn_lstm()

Epoch 1/60
27/27 [==============================] - 7s 84ms/step - loss: 1.6995 - accuracy: 0.3045 - val_loss: 1.8872 - val_accuracy: 0.1887
Epoch 2/60
27/27 [==============================] - 1s 46ms/step - loss: 1.4205 - accuracy: 0.4159 - val_loss: 2.5908 - val_accuracy: 0.1604
Epoch 3/60
27/27 [==============================] - 1s 46ms/step - loss: 1.2870 - accuracy: 0.4763 - val_loss: 2.3493 - val_accuracy: 0.2358
Epoch 4/60
27/27 [==============================] - 1s 44ms/step - loss: 1.1103 - accuracy: 0.5818 - val_loss: 2.4389 - val_accuracy: 0.1840
Epoch 5/60
27/27 [==============================] - 1s 45ms/step - loss: 0.8717 - accuracy: 0.6742 - val_loss: 3.0550 - val_accuracy: 0.1604
Epoch 6/60
27/27 [==============================] - 1s 45ms/step - loss: 0.7438 - accuracy: 0.7204 - val_loss: 2.7011 - val_accuracy: 0.1698
Epoch 7/60
27/27 [==============================] - 1s 46ms/step - loss: 0.5123 - accuracy: 0.8246 - val_loss: 4.0776 - val_accuracy: 0.1604
Epoch 8/60
27

In [130]:
np.save("ravdess_train_data_value", train_data_value)
np.save("ravdess_train_data_target", train_data_target)
np.save("ravdess_test_data_value", test_data_value)
np.save("ravdess_test_data_target", test_data_target)

In [44]:
cnn_lstm()

Epoch 1/60
27/27 [==============================] - 6s 85ms/step - loss: 1.7900 - accuracy: 0.2796 - val_loss: 2.2548 - val_accuracy: 0.1981
Epoch 2/60
27/27 [==============================] - 1s 47ms/step - loss: 1.3975 - accuracy: 0.4242 - val_loss: 2.0174 - val_accuracy: 0.2830
Epoch 3/60
27/27 [==============================] - 1s 45ms/step - loss: 1.3115 - accuracy: 0.4917 - val_loss: 1.7528 - val_accuracy: 0.2689
Epoch 4/60
27/27 [==============================] - 1s 46ms/step - loss: 1.1513 - accuracy: 0.5557 - val_loss: 1.3404 - val_accuracy: 0.4481
Epoch 5/60
27/27 [==============================] - 1s 46ms/step - loss: 0.9531 - accuracy: 0.6422 - val_loss: 1.1671 - val_accuracy: 0.5000
Epoch 6/60
27/27 [==============================] - 1s 47ms/step - loss: 0.8879 - accuracy: 0.6659 - val_loss: 1.3886 - val_accuracy: 0.4481
Epoch 7/60
27/27 [==============================] - 1s 46ms/step - loss: 0.7358 - accuracy: 0.7275 - val_loss: 1.3967 - val_accuracy: 0.4434
Epoch 8/60
27

In [55]:
cnn_lstm()

Epoch 1/60
27/27 [==============================] - 7s 103ms/step - loss: 1.7026 - accuracy: 0.3128 - val_loss: 2.2194 - val_accuracy: 0.1887
Epoch 2/60
27/27 [==============================] - 2s 62ms/step - loss: 1.3634 - accuracy: 0.4550 - val_loss: 2.0203 - val_accuracy: 0.2594
Epoch 3/60
27/27 [==============================] - 1s 53ms/step - loss: 1.1867 - accuracy: 0.5379 - val_loss: 2.0466 - val_accuracy: 0.2264
Epoch 4/60
27/27 [==============================] - 1s 46ms/step - loss: 1.1092 - accuracy: 0.5806 - val_loss: 1.4764 - val_accuracy: 0.4151
Epoch 5/60
27/27 [==============================] - 1s 48ms/step - loss: 0.9163 - accuracy: 0.6457 - val_loss: 1.8733 - val_accuracy: 0.3302
Epoch 6/60
27/27 [==============================] - 1s 46ms/step - loss: 0.8080 - accuracy: 0.6919 - val_loss: 1.4650 - val_accuracy: 0.4104
Epoch 7/60
27/27 [==============================] - 1s 49ms/step - loss: 0.5888 - accuracy: 0.7974 - val_loss: 1.2441 - val_accuracy: 0.5377
Epoch 8/60
2